In [ ]:
all_intersections = []

In [16]:
def find_intersections(dataframe, count=0):
    
    # select the first buffer polygon from the dataframe and create GeoSeries
    first = dataframe.iloc[count, 0]
    
    # find all buffers which intersect chosen polygon (they will be returned as True)
    intersections = dataframe["geometry"].intersects(first)
    
    # save found intersections into separete dataframe
    result = dataframe.loc[intersections]
    
    # if there are intersections, unite polygons and save
    # if number of intersections is 1, it means that polygon intersects only itself
    if result.shape[0] > 1:
            
            #union all intersecting buffers into one polygon 
            union = result["geometry"].unary_union
    
            # create new dataframe and save new polygon in it
            corridor = geopandas.GeoDataFrame(
                data=[[union, "corridor"]], 
                columns = ["geometry", "geo_type"], 
                geometry=[union], 
                crs='epsg:5641')

            # delete processed buffers from initial dataframe and reset index
            dataframe = dataframe[~dataframe.index.isin(result.index)]
            
            #insert found corridor back into dataframe
            dataframe = geopandas.GeoDataFrame(pd.concat([corridor, dataframe], ignore_index=True))
            
            # repeat the same process on the rest of the data in the dataframe         
            find_intersections(dataframe, count=0)
    
    # if number of intersections is 1, it means that polygon intersects only itself
    # now it should be saved in the resulting list
    if result.shape[0] <= 1:
        
        # create new dataframe and save new polygon in it
        if result["polygon"].any():
            separate_polygon = geopandas.GeoDataFrame(data=[[result.iloc[0, 2], result.iloc[0, 1]]], columns = ["geometry", "geo_type"], geometry=[result.iloc[0, 2]], crs='epsg:5641')
        else:
            separate_polygon = geopandas.GeoDataFrame(data=[[result.iloc[0, 0], result.iloc[0, 1]]], columns = ["geometry", "geo_type"], geometry=[result.iloc[0, 0]], crs='epsg:5641')
        
        #insert new dataframe into the resulting list
        all_intersections.append(separate_polygon)
        
        # delete processed polygon from initial dataframe and reset index
        dataframe = dataframe[~dataframe.index.isin(result.index)]
        dataframe = dataframe.reset_index()
        dataframe = dataframe.drop(columns="index")
        
        # if there are still polygons in the datframe, apply the function again
        if count < dataframe.shape[0]:
            find_intersections(dataframe, count = count)
            
    return all_intersections

In [18]:
def create_geodataframe(dataframe):
    if all_intersections:
        all_intersections.clear()
    lst = find_intersections(dataframe)
    print("Intersections found: ", len(lst))
    gdf = geopandas.GeoDataFrame(pd.concat(lst, ignore_index=True))
    return gdf